<a href="https://colab.research.google.com/github/mrkct/cuda-raytracer/blob/master/Relazione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
!rm -rf cuda-raytracer
!git clone https://github.com/mrkct/cuda-raytracer.git
%cd cuda-raytracer
!make all

# Raytracer accelerato via CUDA

Il raytracing è una tecnica utilizzata in computer grafica per il rendering di immagini realistiche. L'idea alla base del raytracing è quella di proiettare dei raggi di luce a partire dall'occhio dello spettatore e, simulando il comportamento dei raggi che interagiscono con gli oggetti nella scena, determinare il colore di un pixel dell'immagine finale. Questa tecnica è raramente utilizzata nella computer grafica in tempo reale a causa della sua pesantezza computazionale perchè richiede di simulare un numero significativo di raggi; almeno uno per ogni pixel dell'immagine finale.
Il calcolo di questi raggi è completamente indipendente tra di loro; questo rende il raytracing rende un candidato ideale alla parallelizzazione.

Il raytracer implementato in questo report è basato su quello  del libro *Raytracing in One Weekend*[1], dove viene implementato in una versione single-thread su CPU. In aggiunta alle feature del libro ho implementato anche il texturing degli oggetti; il video qui sotto è stato prodotto dal mio raytracer generando le immagini dei singoli fotogrammi ed assemblandoli in un video utilizzando `ffmpeg`.

In [ ]:
from IPython.display import Video
Video("demo.mp4")

## Algoritmo per il raytracing
Alla base del raytracing sta il concetto di *raggio*; un raggio è semplicemente una retta in uno spazio tridimensionale, dunque possiamo esprimere i punti su di essa con la semplice formula $P(t) = A + B \cdot t$.

Definiremo questi raggi tramite due punti per cui passano: il primo punto è l'occhio dello spettatore (una costante per tutti i raggi iniziali), l'altro è un punto sulla *vista*. Questa vista è una proiezione nello spazio 3D dell'immagine finale che vorremo generare: dato un punto di coordinate $(x, y)$ nell'immagine finale allora il suo corrispondente nel piano sarà dato da

$$
  \frac{x}{ImageWidth} \cdot ViewWidth \cdot H + \frac{y}{ImageHeight} \cdot ViewHeight \cdot V + LowerLeftCorner
$$

In questa formula $H$ e $V$ sono due vettori unitari che puntano nelle direzioni per muoversi orizzontalmente e verticalmente sul piano.

La dimensione e posizione del piano rispetto alla telecamera ha effetti interessanti sull'immagine finale: tenendo fissa la dimensione del piano ma allontanandolo dallo spettatore provoca un avvicinamento sempre maggiore dei raggi proiettati sul piano che finiscono per concentrarsi un'area più piccola ma in modo più denso: questo è come funziona lo zoom di una telecamera. La distanza di questo piano dalla telecamera è un parametro detto *lunghezza focale*.

**IMMAGINI DEL CAMBIO DI FOCAL LENGTH**

Invece, tenendo fissa la distanza tra telecamera e vista ma variando la dimensione di quest'ultima blabla

Un raytracer, per ogni pixel dell'immagine, proietta un raggio con associato un colore iniziale ed osserva il colore finale del raggio dopo che questo va a collidere con eventuali oggetti nella sua traiettoria dalla telecamera al piano: 

Per accelerare via CUDA questo processo ho diviso l'immagine finale in blocchi quadrati di lato $8$, ogni thread si occupa di un singolo pixel dell'immagine finale e dunque proietta il raggio associato a quel pixel. I thread scrivono il colore finale del loro pixel su un framebuffer allocato utilizzando la managed memory: il motivo di questa scelta è il fatto che la memoria del framebuffer viene spostata tra host e device solamente all'inizio ed al termine del rendering di un fotogramma, dunque seppur non sia complicato gestire questi spostamenti manualmente non ho trovato alcuna differenza tra i due modi.

Prima di iniziare il rendering vengono caricate in *constant memory* alcuni dati che vengono ripetutamente utilizzati da tutti i thread, alcuni esempi di questi dati sono a posizione della telecamera e gli oggetti presenti nella scena.

Una eccezione a questa regola sono i dati delle texture utilizzate da alcuni oggetti nella scena: i valori RGB di queste texture sono salvati in *texture memory*.